In [57]:
from gssutils import *
from databaker.framework import *
import pandas as pd
import datetime

def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

def mid(s, offset, amount):
    return s[offset:offset+amount]

year = right(str(datetime.datetime.now().year),2)

scraper = Scraper('https://www.gov.uk/government/statistics/family-resources-survey-financial-year-201718')
dist = scraper.distribution(title=lambda t: 'Disability data tables (XLS)' in t)
tab = next(t for t in dist.as_databaker() if t.name == '4_2')

In [58]:
tidied_sheets = []
        
cell = tab.excel_ref("B9")

remove = tab.filter('Percentage of people').expand(RIGHT).expand(LEFT).expand(DOWN)
    
year = cell.expand(DOWN).is_not_blank() - remove

gender = cell.shift(2,0).expand(RIGHT).is_not_blank()

observations = gender.fill(DOWN).is_not_blank() - remove

dimensions = [
        #HDimConst('Dimension Name', 'Variable'),
        HDim(year, 'Period', DIRECTLY, LEFT), 
        HDim(gender, 'Gender', DIRECTLY, ABOVE), 
        HDimConst('Age Group', 'All people'),
        HDimConst('Measure type','Count'),
        HDimConst('Unit','People (Millions)')    
]
    
c1 = ConversionSegment(observations, dimensions, processTIMEUNIT=True)
savepreviewhtml(c1, fname="Preview.html")

tablepart '4_2' written #injblock1015
javascript calculated


C:\Users\Red\Documents\COGS\Disability\DWP-Family-Resources-Survey-Disability\Preview.html

In [59]:
new_table = c1.topandas()
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Period'] = new_table['Period'].map(
    lambda x: f'gregorian-interval/{left(x,2) + right(x,2)}-03-31T00:00:00/P1Y')
new_table['Age Group'] = new_table['Age Group'].map(
    lambda x: left(x, len(x) - 1) if x.endswith('1') else x)
new_table = new_table[['Period','Gender','Age Group','Measure type','Value','Unit']]
new_table

,Period,Gender,Age Group,Measure type,Value,Unit
0,gregorian-interval/2008-03-31T00:00:00/P1Y,Males,All people,Count,5,People (Millions)
1,gregorian-interval/2008-03-31T00:00:00/P1Y,Females,All people,Count,5,People (Millions)
2,gregorian-interval/2009-03-31T00:00:00/P1Y,Males,All people,Count,5,People (Millions)
3,gregorian-interval/2009-03-31T00:00:00/P1Y,Females,All people,Count,6,People (Millions)
4,gregorian-interval/2010-03-31T00:00:00/P1Y,Males,All people,Count,5,People (Millions)
5,gregorian-interval/2010-03-31T00:00:00/P1Y,Females,All people,Count,6,People (Millions)
6,gregorian-interval/2011-03-31T00:00:00/P1Y,Males,All people,Count,5,People (Millions)
7,gregorian-interval/2011-03-31T00:00:00/P1Y,Females,All people,Count,6,People (Millions)
8,gregorian-interval/2012-03-31T00:00:00/P1Y,Males,All people,Count,5,People (Millions)
9,gregorian-interval/2012-03-31T00:00:00/P1Y,Females,All people,Count,6,People (Millions)


In [60]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

TAB_NAME = 'Disability-prevalence-by-gender-2007-08-to-2017-18'

new_table.drop_duplicates().to_csv(destinationFolder / f'{TAB_NAME}.csv', index = False)